In [ ]:
%pip install azure-ai-projects
%pip install azure-ai-inference[prompts]
%pip install azure-identity
%pip install azure-search-documents
%pip install pandas
%pip install python-dotenv
%pip install opentelemetry-api

In [2]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"aifoundry-apim-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "eastus"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

ai_studio_hub_name = 'hub'
ai_studio_project_name = 'project'
storage_account_name = 'storage'
keyvault_name = 'keyvault'
container_registry_name = 'registry'

openai_embeddings_deployment_name = "text-embedding-ada-002"
openai_embeddings_model_name = "text-embedding-ada-002"
openai_embeddings_model_version = "2"
searchservice_resource_name = "search"
searchservice_sku = "standard"
searchindex_name = "example-index"

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())
    
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "aiStudioHubName": { "value": ai_studio_hub_name },
    "aiStudioProjectName": { "value": ai_studio_project_name },
    "storageAccountName": { "value": storage_account_name }, 
    "keyVaultName": { "value": keyvault_name },
    "containerRegistryName": { "value": container_registry_name },
    "openAIEmbeddingsDeploymentName": { "value": openai_embeddings_deployment_name},
    "openAIEmbeddingsModelName": { "value": openai_embeddings_model_name},
    "openAIEmbeddingsModelVersion": { "value": openai_embeddings_model_version},
    "searchServiceName": { "value": searchservice_resource_name},
    "searchServiceSku": { "value": searchservice_sku}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)

In [4]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.projectConnectionString.value -o tsv
project_connection_string = deployment_stdout.n
print("👉🏻 Project Connection String: ", project_connection_string)

👉🏻 Project Connection String:  westeurope.api.azureml.ms;6e37307e-394c-478a-8404-4e441b3dfc1d;aifoundry-apim-genai;project-vhrjph3teihp2
